In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid", {'axes.grid' : False})
from keras.backend.tensorflow_backend import set_session
import keras, sys, time, warnings
from keras.models import *
from keras.layers import *
from keras.optimizers import Adam

/Users/adishrao/anaconda3/lib/python3.6/site-packages/matplotlib/__init__.py:886: MatplotlibDeprecationWarning: 
examples.directory is deprecated; in the future, examples will be found relative to the 'datapath' directory.
  "found relative to the 'datapath' directory.".format(key))
Using TensorFlow backend.


In [2]:
dataset_dir = 'small_size_data'
label_dir_train = dataset_dir + '/train/labels/'
img_dir_train = dataset_dir + '/train/images/'
label_dir_validation = dataset_dir + '/validation/labels/'
img_dir_validation = dataset_dir + '/validation/images/'

In [3]:
labels_train = sorted(os.listdir(label_dir_train))
images_train = sorted(os.listdir(img_dir_train))

labels_validation = sorted(os.listdir(label_dir_validation))
images_validation = sorted(os.listdir(img_dir_validation))


# mac specific .DS_STORE 
if images_train[0] == '.DS_Store':
    images_train.remove('.DS_Store')
if images_validation[0] == '.DS_Store':
    images_validation.remove('.DS_Store')
if labels_train[0] == '.npy':
    labels_train.pop(0)
if labels_validation[0] == '.npy':
    labels_validation.pop(0)

In [4]:
width, height = 96,160
nc = 3
DATASIZE = 400

X_train = []
Y_train = []
X_val = []
Y_val = []
ik=0


for img_path, lbl_path in list(zip(images_train,labels_train))[:DATASIZE]:
    if img_path[:12] != lbl_path[:12]:
        print('ERROR. Check folder organisation')
        break
        
for img_path, lbl_path in list(zip(images_validation,labels_validation))[:DATASIZE]:
    if img_path[:12] != lbl_path[:12]:
        print('ERROR. Check folder organisation')
        break

for img_path, lbl_path in list(zip(images_train,labels_train))[:]:
    try:
        # load image and label, resize, append
        img = cv2.imread(img_dir_train + img_path)
        img = np.float32(img) / 255 
        X_train.append(img)

        lbl = np.load(label_dir_train + lbl_path)
        Y_train.append(lbl)
    except Exception as e: 
        print(e)
        print(img_path,lbl_path)
        ik+=1
print("i",ik)
X_train, Y_train = np.array(X_train) , np.array(Y_train)
ik = 0
for img_path, lbl_path in list(zip(images_validation,labels_validation))[:]:
    try:
        # load image and label, resize, append
        img = cv2.imread(img_dir_validation + img_path)
        img = np.float32(img) / 255 # - 1 #averagin values. sohuld we remove?
        X_val.append(img)

        lbl = np.load(label_dir_validation + lbl_path)
        Y_val.append(lbl)
    except Exception as e: 
        print(e)
        print(img_path,lbl_path)
        ik+=1
print("i",ik)
X_val, Y_val = np.array(X_val) , np.array(Y_val)

i 0
i 0


In [12]:
def find_middle(X):
    for i,r in enumerate(X):
        if 1 in r:
            break
    rowstart  = i
    for i,r in enumerate(X[::-1]):
        if 1 in r:
            break
    rowend  = 159 - i
    
    for j in range(96):
        if 1 in X[:,j]:
            break
    colstart = j
    colend = 0
    for r in X:
        for index,j in enumerate(r):
            if j == 1:
                if index>colend:
                    colend = index
    return (rowstart,rowend,colstart,colend)

In [13]:
xy_list = []
i = 0
for predict in Y_train:
    rowstart,rowend,colstart,colend = find_middle(predict)
    midY, midX = int((rowstart+rowend)/2), int((colstart+colend)/2)
    xy_list.append((midY,rowstart,rowend))

In [14]:
for predict in Y_val:
    rowstart,rowend,colstart,colend = find_middle(predict)
    midY, midX = int((rowstart+rowend)/2), int((colstart+colend)/2)
    xy_list.append((midY,rowstart,rowend))

In [15]:
maxi = 0
for (x,y,z) in xy_list:
    if (z-y) > maxi:
        maxi = z-y

In [16]:
print(maxi)

68


In [11]:
xy_list

[(95, 88, 103),
 (96, 88, 104),
 (96, 89, 104),
 (95, 87, 103),
 (95, 87, 103),
 (92, 84, 100),
 (92, 84, 100),
 (92, 84, 100),
 (92, 84, 100),
 (92, 84, 100),
 (71, 64, 78),
 (95, 88, 103),
 (96, 88, 104),
 (92, 84, 100),
 (91, 84, 99),
 (92, 84, 100),
 (90, 83, 98),
 (88, 81, 96),
 (83, 76, 91),
 (56, 50, 62),
 (56, 50, 62),
 (88, 81, 95),
 (94, 87, 102),
 (94, 87, 102),
 (94, 87, 102),
 (94, 87, 102),
 (94, 87, 102),
 (95, 87, 103),
 (94, 87, 102),
 (75, 69, 82),
 (75, 70, 81),
 (73, 66, 80),
 (91, 84, 98),
 (91, 84, 98),
 (91, 84, 98),
 (91, 84, 98),
 (59, 53, 66),
 (81, 74, 88),
 (66, 59, 73),
 (98, 90, 106),
 (98, 91, 106),
 (66, 59, 73),
 (91, 84, 99),
 (93, 86, 101),
 (92, 85, 100),
 (88, 82, 95),
 (89, 82, 96),
 (77, 71, 83),
 (76, 72, 81),
 (76, 71, 81),
 (76, 70, 82),
 (92, 86, 99),
 (92, 86, 99),
 (92, 85, 99),
 (92, 85, 99),
 (92, 85, 99),
 (82, 76, 89),
 (94, 88, 101),
 (95, 88, 102),
 (93, 86, 101),
 (94, 87, 101),
 (93, 86, 101),
 (90, 83, 97),
 (89, 84, 95),
 (99, 92, 